In [ ]:
!conda create -n codex python=3.7
! conda activate codex
! git clone https://github.com/openai/human-eval
! pip install -e human-eval
# Set the working directory
%cd /content/human-eval
!pip install jsonlines

In [27]:
from human_eval.data import write_jsonl, read_problems
import pandas as pd
import time
import jsonlines
from openai import OpenAI
def generate_one_completion(prompt):
  client = OpenAI(api_key="sk-4d5e3e6f328e47cb93fc46dddac2b450", base_url="https://api.deepseek.com/v1")
  response = client.chat.completions.create(
    model="deepseek-coder",messages=[
        {"role": "system",
         "content": "You are an intelligent programmer. You must complete the python function given to you by the user. And you must follow the format they present when giving your answer!"},
        {"role": "user", "content": prompt},
    ],
    max_tokens=1048,
    temperature=0.0)

  return(response.choices[0].message.content)

In [33]:
import pandas as pd
import time
import jsonlines
import numpy as np

if __name__ == '__main__':
    jsonObj = pd.read_json(path_or_buf="https://github.com/openai/human-eval/raw/master/data/HumanEval.jsonl.gz", lines=True)
    command = 0
    i = 0
    durationTimes = []
    with jsonlines.open('Deepcoder.jsonl', mode='w') as writer:
        while i <  164:
            if command < 4:
                start_time = time.time()
                response = generate_one_completion(jsonObj["prompt"][i])
                duration = time.time() - start_time
                response = response.replace("```python\n", "")
                response = response.replace("\n```", "")
                durationTimes.append(duration)
                completion = {'task_id': jsonObj["task_id"][i], 'completion': response}
                writer.write(completion)
                command = command + 1
                i = i + 1
            else:
                command = 0
                time.sleep(90)
    print(np.sum(duration))
    print(np.average(duration))

13.302963972091675
13.302963972091675


In [34]:
!python3 human_eval/evaluate_functional_correctness.py /Deepcoder.jsonl

Reading samples...
165it [00:00, 16139.09it/s]
Running test suites...
100% 165/165 [00:02<00:00, 74.88it/s]
Writing results to /Deepcoder.jsonl_results.jsonl...
100% 165/165 [00:00<00:00, 38612.96it/s]
{'pass@1': 0.17682926829268292}
